### Import

In [107]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' #GPU 사용시 초기 메모리를 작게하여 OOM 방지하기 위한 목적
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, concatenate
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import numpy as np

## 트랜스포머

In [108]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' #GPU 사용시 초기 메모리를 작게하여 OOM 방지하기 위한 목적
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, concatenate
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import numpy as np

In [109]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

        # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
        sines = tf.math.sin(angle_rads[:, 0::2])
#         print(sines.shape)
        # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
        cosines = tf.math.cos(angle_rads[:, 1::2])
#         print(cosines.shape)
        pos_encoding = tf.concat([sines, cosines], axis=-1)
#         print(pos_encoding.shape)  #  (50, 128)
#         print(pos_encoding[1])
        
        pos_encoding = pos_encoding[tf.newaxis, ...]
        print(pos_encoding.shape) # (1, 50, 128)
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
              #  (1, 10, 128)                 (1, 10, 128)

def scaled_dot_product_attention(query, key, value, mask):
    # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    # padding_mask : (batch_size, 1, 1, key의 문장 길이)

    # Q와 K의 곱. 어텐션 스코어 행렬.
    matmul_qk = tf.matmul(query, key, transpose_b=True)  # (1,8,40,32)(1,8,32,40) => (1,8,40,40)

    # 스케일링
    # dk의 루트값으로 나눠준다.
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
    # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
    if mask is not None:
        logits += (mask * -1e9)

    # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
    # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    output = tf.matmul(attention_weights, value)  # (1,8,40,40)(1,8,40,32) => (1,8,40,32)

    return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        # d_model을 num_heads로 나눈 값.
        # 논문 기준 : 64
        self.depth = d_model // self.num_heads

        # WQ, WK, WV에 해당하는 밀집층 정의
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        # WO에 해당하는 밀집층 정의
        self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # 1. WQ, WK, WV에 해당하는 밀집층 지나기
        # q : (batch_size, query의 문장 길이, d_model)
        # k : (batch_size, key의 문장 길이, d_model)
        # v : (batch_size, value의 문장 길이, d_model)
        # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
        #print('query.shape', query.shape)
        #print('key.shape', key.shape)
        #print('value.shape', value.shape)  # (40,256)
        query = self.query_dense(query)    # (40,256)(256,256)
        key = self.key_dense(key)
        value = self.value_dense(value)    # (40,256)

        # 2. 헤드 나누기
        # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
        # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
        # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        
        #print('query.shape', query.shape)  # (1,8,40,32)
        #print('key.shape', key.shape)
        #print('value.shape', value.shape)  # (1,8,40,32)        

        # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
        # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
        # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3]) # (1,40,8,32) 

        # 4. 헤드 연결(concatenate)하기
        # (batch_size, query의 문장 길이, d_model)
        concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (1,40,256) 

        # 5. WO에 해당하는 밀집층 지나기
        # (batch_size, query의 문장 길이, d_model)
        outputs = self.dense(concat_attention)  # (1,40,256)(256,256) => (1,40,256)

        return outputs
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, key의 문장 길이)
    return mask[:, tf.newaxis, tf.newaxis, :]

def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

    # 인코더는 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
    attention = MultiHeadAttention(
        d_model, num_heads, name="attention")({
            'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
            'mask': padding_mask # 패딩 마스크 사용
        })

    # 드롭아웃 + 잔차 연결과 층 정규화
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(inputs + attention)

    # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
    outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention) # (1,40,256)(256,512)=>(1,40,512)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs) # (40,512)(512,256)=>(1,40,256)

    # 드롭아웃 + 잔차 연결과 층 정규화
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    # 인코더는 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    # 포지셔널 인코딩 + 드롭아웃
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    # 인코더를 num_layers개 쌓기
    for i in range(num_layers):
      outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
          dropout=dropout, name="encoder_layer_{}".format(i),
      )([outputs, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
    return tf.maximum(look_ahead_mask, padding_mask)
#     return look_ahead_mask

def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

    # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
    attention1 = MultiHeadAttention(
        d_model, num_heads, name="attention_1")(inputs={
            'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
            'mask': look_ahead_mask # 룩어헤드 마스크
        })

    # 잔차 연결과 층 정규화
    attention1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention1 + inputs)

    # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
    attention2 = MultiHeadAttention(
        d_model, num_heads, name="attention_2")(inputs={
            'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
            'mask': padding_mask # 패딩 마스크
        })

    # 드롭아웃 + 잔차 연결과 층 정규화
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention2 + attention1)

    # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
    outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 드롭아웃 + 잔차 연결과 층 정규화
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

    # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 포지셔널 인코딩 + 드롭아웃
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    # 디코더를 num_layers개 쌓기
    for i in range(num_layers):
        outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
            dropout=dropout, name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)
    
def transformer(vocab_size, num_layers, dff,
                  d_model, num_heads, dropout,
                  name="transformer"):

    # 인코더의 입력
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    # 디코더의 입력
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    # 인코더의 패딩 마스크
    enc_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='enc_padding_mask')(inputs)

    # 디코더의 룩어헤드 마스크(첫번째 서브층)
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask, output_shape=(1, None, None),
        name='look_ahead_mask')(dec_inputs)

    # 디코더의 패딩 마스크(두번째 서브층)
    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='dec_padding_mask')(inputs)

    # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
    enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
        d_model=d_model, num_heads=num_heads, dropout=dropout,
    )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

    # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
    dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
        d_model=d_model, num_heads=num_heads, dropout=dropout,
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    # 다음 단어 예측을 위한 출력층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)


In [17]:
import tensorflow_datasets as tfds
import re
from tensorflow.keras.callbacks import EarlyStopping
import konlpy
from konlpy.tag import *
konlpy.__version__

'0.5.2'

In [21]:
!pip install tokenizers

In [110]:
from tokenizers import BertWordPieceTokenizer

# Initialize an empty BERT tokenizer
btokenizer = BertWordPieceTokenizer(
  clean_text=False,
  handle_chinese_chars=False,
  strip_accents=False,
  lowercase=True,
)

# prepare text files to train vocab on them
files = [os.path.join(base_path, 'ko_en_valid_small.txt')]

# train BERT tokenizer
btokenizer.train(
  files,
  vocab_size=100,
  min_frequency=2,
  show_progress=True,
  special_tokens=['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'],
  limit_alphabet=1000,
  wordpieces_prefix="##"
)

# save the vocab
btokenizer.save('bert_tokenizer_simple.tkz')


In [23]:
!cp  ./bert_tokenizer_simple.tkz ./drive/MyDrive/ColabData/

In [111]:
def convert_text(str_list):
    ret = []
    for tmp in str_list:
        sentence = tmp
        # 구두점에 대해서 띄어쓰기
        # ex) 12시 땡! -> 12시 땡 !
        sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
        sentence = sentence.strip()
        ret.append(sentence)
    return ret

In [112]:
#filenames 들로 부터 데이터를 읽음
def read_source(filenames):
    que = []
    ans = []
    for filename in filenames:
        with open(filename, 'r', encoding='utf8') as fp:
            lines = fp.readlines()
            for line in lines:
                line = line.strip()
                items = line.split('|')
                que.append(items[0].strip())
                ans.append(items[1].strip())
    return (que, ans)
                
#questions, answers = read_source([src_file_path])
#print(len(questions), len(answers))
#print(questions[:2], answers[:2])

In [113]:
#filenames 들로 부터 데이터를 읽음
def read_source_translation(filenames):
    que = []
    ans = []
    for filename in filenames:
        with open(filename, 'r', encoding='utf8') as fp:
            lines = fp.readlines()
            for line in lines:
                line = line.strip()
                items = line.split('|')
                que.append(items[1].strip())
                ans.append(items[2].strip())
    return (que, ans)

questions, answers = read_source_translation([os.path.join(base_path, 'ko_en_valid_small.txt')])
print(len(questions), len(answers))
print(questions[:2])                
print(answers[:2])

5000 5000
['아라미드 원단은 아라미드 섬유를 포함하고, 계산식 1에 나타낸 바와 같은 커버팩터가 1,000 ~ 2,000이다.경사 섬도 및 위사 섬도는 각각 200 ~ 1,000 데니어이다.경사밀도 및 위사밀도는 각각 20 ~ 55 th/inch이다.미국재료시험협회규격 ASTM D 4032 방법에 따른 강연도가 1.5 kgf 이하이다.국제표준화기구 ISO 2286-3 방법으로 측정한 후도가 0.10 mm ~ 0.45 mm이다.미국재료시험협회규격 ASTM D 5034 방법으로 측정한 인장강도가 150 kgf/10mm 이상이다.미국재료시험협회규격 ASTM D 1683 방법으로 측정한 봉목강도가 150 kgf/20mm 이상이다.차량용 에어백은 아라미드 원단을 포함한다.에어백은 프론탈용 에어백, 사이드 커튼형 에어백, 또는 범퍼 타입 에어백이다.', '위치 관리 서버(110)는 RFID 태그 인식결과에 따라 환자와 의료진의 위치를 관리한다.환경 제어 서버(120)는 개인 식별정보와 매칭하여 환자별 치료 환경정보를 저장한다.상기 환경 제어 서버는 상기 환자의 치료 환경정보에 따라 병실 또는 치료실의 환경을 제어한다.환자 감시 서버(130)는 상기 환자의 위치 정보를 기반으로, 병원 내 카메라들을 구동하여 상기 환자의 상태를 감시한다.']
['An aramid fabric for an air bag(124) has an aramid fiber.The fineness of warp and weft is each 200-1000 denier.The density of the warp and weft is each 20-55th/inch.The stiffness of the aramid fabric is 1.5 kgf or less according to American Society for Testing Materials(ASTM) D4032 method.The tensile strength of the aramid fabric is 150 kgf/10mm or g

In [71]:
# 서브워드텍스트인코더를 사용하여 질문, 답변 데이터로부터 단어 집합(Vocabulary) 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [28]:
print(btokenizer.get_vocab_size())
print(btokenizer.encode('[CLS]').ids)
print(btokenizer.encode('[SEP]').ids)
#tokenizer.

1890
[2]
[3]


In [114]:
# 시작 토큰과 종료 토큰에 대한 정수 부여.

#START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
#VOCAB_SIZE = tokenizer.vocab_size + 2

#For bert tokenizer
START_TOKEN, END_TOKEN = [btokenizer.encode('[CLS]').ids[0]], [btokenizer.encode('[SEP]').ids[0]]
VOCAB_SIZE = btokenizer.get_vocab_size()
VOCAB_SIZE = 2000

In [30]:
#mq = max([len(i) for i in data_q])
#ma = max([len(i) for i in data_a])
#print(mq, ma)

In [115]:
data_q = convert_text(questions)
data_a = convert_text(answers)

print( max([len(btokenizer.encode(s).ids) for s in data_q]))
print( max([len(btokenizer.encode(s).ids) for s in data_a]))

843
1864


In [116]:
# 최대 길이를 40으로 정의
MAX_LENGTH = 2000

data_q = convert_text(questions)
data_a = convert_text(answers)

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []

    for (sentence1, sentence2) in zip(inputs, outputs):
        # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
        #sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        #sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

        sentence1 = START_TOKEN + btokenizer.encode(sentence1).ids + END_TOKEN
        sentence2 = START_TOKEN + btokenizer.encode(sentence2).ids + END_TOKEN

        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)

    # 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

    return tokenized_inputs, tokenized_outputs
data_q, data_a = tokenize_and_filter(data_q, data_a)

In [117]:
print(START_TOKEN, END_TOKEN)
print(type(data_q[0][0]), data_q[0], data_q[0].shape)
print(data_a[0], data_a[0].shape)

[2] [3]
<class 'numpy.int32'> [   2  589 1047 ...    0    0    0] (2000,)
[   2   37 1022 ...    0    0    0] (2000,)


In [118]:
BATCH_SIZE = 2
BUFFER_SIZE = 1000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': data_q,
        'dec_inputs': data_a[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': data_a[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [119]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      #START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
      START_TOKEN + btokenizer.encode(sentence).ids + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      print('pid', predicted_id)
      break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence


In [122]:
D_MODEL = 32
NUM_LAYERS = 2
NUM_HEADS = 4
DFF = 32
DROPOUT = 0.1

#임시 테스트
#VOCAB_SIZE = 500

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 2000, 32)
(1, 2000, 32)


In [123]:
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 32)     76928       inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [124]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

earlystop_callback = EarlyStopping(monitor='accuracy', min_delta=0.0001,patience=10)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [50]:
!mkdir sg_ckpt

mkdir: cannot create directory ‘sg_ckpt’: File exists


In [125]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/


Epoch 1/3
2500/2500 [==============================] - 326s 125ms/step - loss: 1.1736 - accuracy: 0.0312
Epoch 2/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.6337 - accuracy: 0.0677
Epoch 3/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.6148 - accuracy: 0.0717
Input: 광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.
Output: a which 유동which 유동which 유동헤드which 유동which 유동which 유동헤드모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 유동선 모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 유동선 모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청와이어선 모듈은 청와이어선 모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모듈은 청모

In [126]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/


Epoch 1/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.5980 - accuracy: 0.0772
Epoch 2/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.5632 - accuracy: 0.0873
Epoch 3/3
2500/2500 [==============================] - 301s 121ms/step - loss: 0.5333 - accuracy: 0.0960
pid tf.Tensor([[3]], shape=(1, 1), dtype=int32)
Input: 광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.
Output: a which or 가스 In 형성청지지부head 모듈은 head which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드지지부대하여 와이어선택된 모듈은 is 에 레이저 The ), ed 곡모듈은 or 가스 In 형성청지지부유동protruded 100와이어청선 is 에 레이저 The from head 제 가스 청In 모듈은 헤드from 유동head 제 가스 청In 모듈은 헤드제1 유동ed 곡모듈은 된 모듈은 대하여 선 지지부유동protruded 100와이어청선 모듈은 헤드제1 수지 ed 곡모듈은 된 모듈은 대하여 선 지지부가스 유동헤드모듈은 which 유동헤드which 유동헤드which 유동which 유동헤드which 유동헤드which 유동헤드which 유동헤드which

In [127]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/


Epoch 1/3
2500/2500 [==============================] - 301s 121ms/step - loss: 0.5151 - accuracy: 0.1013
Epoch 2/3
2500/2500 [==============================] - 301s 121ms/step - loss: 0.5018 - accuracy: 0.1051
Epoch 3/3
2500/2500 [==============================] - 301s 120ms/step - loss: 0.4919 - accuracy: 0.1080
pid tf.Tensor([[3]], shape=(1, 1), dtype=int32)
Input: 광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.
Output: a which 된 모듈은 선 가스 선택된 선택된 모듈은 대하여 선 지지부가스 유동side 유동지지부선 is 에 레이저 The from head 제 가스 청In 모듈은 헤드from 유동ed 곡모듈은 된 모듈은 선택된 모듈은 대하여 선 지지부가스 유동side 유동지지부선 is 에 레이저 The ), which 된 모듈은 대하여 선 지지부가스 유동side 유동지지부선 is 에 레이저 The from head 제 가스 청In 모듈은 헤드제1 유동ed 곡모듈은 된 모듈은 선택된 모듈은 대하여 선 지지부가스 유동side 유동지지부선 is 에 레이저 The from head 제 가스 청In 모듈은 헤드제1 유동ed 곡모듈은 된 모듈은 선택된 모듈은 대하여 선 지지부가스 유동side 유동지지부선 is 에 레이저 The ), ed 곡모듈은 된 모듈은 선 모듈은 청ed 가스 ed 곡모듈은 된 모듈은 선 모듈은 청ed 가스 ed 곡모듈은 된 모듈은 선택된 모듈은 대하여 선 지지부가스 유동side 유동지지부선 is 에

In [128]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/


Epoch 1/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.4841 - accuracy: 0.1102
Epoch 2/3
2500/2500 [==============================] - 305s 122ms/step - loss: 0.4778 - accuracy: 0.1120
Epoch 3/3
2500/2500 [==============================] - 306s 123ms/step - loss: 0.4726 - accuracy: 0.1135
Input: 광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.
Output: a which 된 선 가스 청모듈은 대하여 선 지지부가스 유동or 가스 유동선 와이어선택된 선택된 naor 가스 유동선 와이어선택된 선택된 naor 가스 유동선 와이어선택된 선택된 naor 가스 유동선 청가스 선택된 side 유동지지부선 is 에 레이저 The ), ed 곡모듈은 된 선 와이어선택된 naor 가스 유동선 와이어선택된 선택된 naor 가스 유동선 와이어선택된 선택된 naor 가스 유동선 와이어선택된 선택된 naor 가스 유동선 와이어선택된 is 에 레이저 The from head 제 가스 청In 모듈은 헤드from 유동ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모

In [129]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/

Epoch 1/3
2500/2500 [==============================] - 306s 123ms/step - loss: 0.4683 - accuracy: 0.1147
Epoch 2/3
2500/2500 [==============================] - 306s 122ms/step - loss: 0.4646 - accuracy: 0.1157
Epoch 3/3
2500/2500 [==============================] - 306s 122ms/step - loss: 0.4610 - accuracy: 0.1168
pid tf.Tensor([[3]], shape=(1, 1), dtype=int32)
Input: 광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.
Output: a which 된 선 와이어선택된 모듈은 is 에 레이저 The from 유동대하여 선택된 bar 헤드모듈은 head which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동which 유동헤드which 유동헤드whic

In [130]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/

Epoch 1/3
2500/2500 [==============================] - 306s 122ms/step - loss: 0.4581 - accuracy: 0.1176
Epoch 2/3
2500/2500 [==============================] - 306s 122ms/step - loss: 0.4549 - accuracy: 0.1184
Epoch 3/3
2500/2500 [==============================] - 304s 121ms/step - loss: 0.4524 - accuracy: 0.1191
pid tf.Tensor([[3]], shape=(1, 1), dtype=int32)
Input: 광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.
Output: a which 된 선 모듈은 대하여 선 모듈은 유동bar 선 or 가스 유동선 와이어대하여 선 모듈은 head which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드지지부유동protruded side 유동지지부선 is 에 location 레이저 The 는 which 유동헤드which 유동which 유동헤드which 유동which 유동헤드which 유동which 유동헤드which 유동which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동which 유동헤드which 유동which 유동헤드which 유동헤드which 유동헤드which 유동which 유동헤드which 유동which 유동헤드which 유동which

In [131]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/

Epoch 1/3
2500/2500 [==============================] - 301s 120ms/step - loss: 0.4500 - accuracy: 0.1197
Epoch 2/3
2500/2500 [==============================] - 301s 120ms/step - loss: 0.4480 - accuracy: 0.1204
Epoch 3/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.4461 - accuracy: 0.1209
Input: 광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.
Output: a which 된 선 모듈은 형성모듈은 청와이어선 모듈은 청is in 단말레이저 The from 유동대하여 선택된 bar 헤드모듈은 head which 유동헤드which 된 모듈은 형성청와이어선 모듈은 is in location 레이저 The ), which 된 선 모듈은 형성청와이어선 지지부유동protruded side 유동지지부선 is in location 레이저 The from head which 유동헤드which 된 모듈은 형성청와이어선 모듈은 헤드from 유동ed 곡모듈은 된 모듈은 형성청와이어선 모듈은 헤드from 유동ed 곡모듈은 된 모듈은 형성청와이어선 모듈은 헤드from 유동ed 곡모듈은 된 모듈은 형성청와이어선 모듈은 헤드from 유동ed 곡모듈은 된 모듈은 형성청와이어선 모듈은 헤드제1 유동ed 곡모듈은 된 모듈은 형성청와이어선 모듈은 헤드from 유동ed 곡모듈은 된 지지부헤드모듈은 제1 수지 ed 곡모듈은 된 bar 청수지 와이어대하여 모듈은 제1 수지 ed 곡모듈은 된 bar 청수지 와이어대하여 모듈은 제1 수지 ed 곡모듈은 된 bar 청수지 와이어대하여

In [132]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/

Epoch 1/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.4439 - accuracy: 0.1215
Epoch 2/3
2500/2500 [==============================] - 303s 121ms/step - loss: 0.4424 - accuracy: 0.1220
Epoch 3/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.4409 - accuracy: 0.1223
Input: 광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.
Output: a which 된 선 모듈은 형성모듈은 청와이어선 모듈은 청is in 레이저 The from head which 유동헤드which 된 모듈은 형성청와이어선 모듈은 청is in location 레이저 The 는 which 유동헤드which 된 모듈은 형성청와이어선 모듈은 is in location 레이저 The 는 which 유동헤드which 된 모듈은 형성청와이어선 모듈은 head which 유동헤드which 된 모듈은 형성청와이어선 모듈은 청is in location 레이저 The 는 which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드w

In [133]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/

Epoch 1/3
2500/2500 [==============================] - 303s 121ms/step - loss: 0.4393 - accuracy: 0.1228
Epoch 2/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.4380 - accuracy: 0.1232
Epoch 3/3
2500/2500 [==============================] - 302s 121ms/step - loss: 0.4366 - accuracy: 0.1235
pid tf.Tensor([[3]], shape=(1, 1), dtype=int32)
Input: 광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.
Output: a which 유동which 유동헤드which 유동which 유동헤드which 유동which 유동헤드지지부유동protruded 100와이어청선 is in 단말레이저 The ), which 유동which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드which 유동헤드w

In [ ]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystop_callback])
output = predict("광학 시트가 몸체부와 돌출부(220)를 포함한다.상기 돌출부가 상기 몸체부 위에 소정 두께만큼 돌출된다.스캐터링부(223)가 상기 돌출부의 소정 부위에 형성된다.상기 스캐터링부의 표면이 거칠게 형성된다.")
model.save_weights('./sg_ckpt/sg_ckpt')
!cp -R  ./sg_ckpt/* ./drive/MyDrive/ColabData/

Epoch 1/3
2500/2500 [==============================] - 303s 121ms/step - loss: 0.4353 - accuracy: 0.1240
Epoch 2/3
1087/2500 [============>.................] - ETA: 2:50 - loss: 0.4348 - accuracy: 0.1233